In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import gensim

In [3]:
from gensim.models import Word2Vec, KeyedVectors

In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import matplotlib.pyplot as plt
import nltk

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
data = pd.read_csv("data/news_class.csv")

In [7]:
df = data.copy() # to be safe and avoid errors

In [8]:
df = df.loc[:,["data_id" , "content" , "category_level_1" , "category_level_2"]]

In [9]:
def labeler(dataframe_column):
    encoder=LabelEncoder()
    
    labels = encoder.fit_transform(dataframe_column)
    print(encoder.classes_)
    print("We did it boys , labels have been created")
    
    return(pd.DataFrame(labels))

In [10]:
# Function for removing ASCII characters
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

# Function for converting to lower case
def make_lower_case(text):
    return text.lower()

# Function for removing stop words
def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

# Function for removing html
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

# Function for removing punctuation
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

def lemm_text(text):
    lemm=WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    tokens = text
    return ' '.join([lemm.lemmatize(t) for t in tokens])

In [11]:
df['Cleaned'] = df['content'].apply(_removeNonAscii)
df['Cleaned'] = df.Cleaned.apply(func = make_lower_case)
df['Cleaned'] = df.Cleaned.apply(func = remove_stop_words)
df['Cleaned'] = df.Cleaned.apply(func = remove_punctuation)
df['Cleaned'] = df.Cleaned.apply(func = remove_html)
df['Cleaned'] = df.Cleaned.apply(func = lemm_text)
df = df.drop('content' , axis =1)

In [12]:
io = df.copy()

In [13]:
io = io.drop([374]).reset_index(drop=True)
io = io.drop([6527]).reset_index(drop=True)


corpus_full = []
for words in io['Cleaned']:
    corpus_full.append(words.split())  

In [14]:
def vectorizer():
    
    
    # Creating a list for storing the vectors ('Description' into vectors)
    #global word_embeddings
    word_embeddings = []
    i = 0
    # Reading the each 'Description'
    for line in io['Cleaned']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in model.wv.key_to_index:
                count += 1
                if avgword2vec is None:
                    avgword2vec = model.wv[word]
                else:
                    avgword2vec = avgword2vec + model.wv[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
            word_embeddings.append(avgword2vec)
        else:
            print("I found it , the error occurs at line:" , i)
          
        i +=1 

    return(pd.DataFrame(word_embeddings))  # Returning our Data as a Dataframe (aesthetic reasons)

In [15]:
model = Word2Vec.load("word2vec.model3")

In [16]:
vect = vectorizer()

In [17]:
y1 = labeler(io["category_level_1"])

['arts, culture, entertainment and media' 'conflict, war and peace'
 'crime, law and justice' 'disaster, accident and emergency incident'
 'economy, business and finance' 'education' 'environment' 'health'
 'human interest' 'labour' 'lifestyle and leisure' 'politics'
 'religion and belief' 'science and technology' 'society' 'sport'
 'weather']
We did it boys , labels have been created


In [18]:
X = vect.copy()

In [19]:
X

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.010030,-0.072825,-0.051542,0.033568,0.084110,0.027619,-0.071665,0.015907,-0.006390,0.152007,...,-0.087481,0.150550,0.010014,0.063487,0.218890,0.049933,0.014407,-0.129096,0.003949,-0.131842
1,0.071578,-0.083879,-0.080275,0.015637,0.068045,0.007964,-0.054446,0.003107,0.003455,0.160702,...,-0.032729,0.109924,0.010929,0.027677,0.182586,0.082423,-0.014971,-0.097462,-0.027377,-0.059375
2,0.028660,-0.003605,-0.043669,0.050716,0.109462,0.030837,-0.033665,0.022952,-0.026909,0.246786,...,-0.016610,0.091539,-0.029639,0.123434,0.160820,0.140771,-0.031121,-0.071247,-0.039745,-0.078560
3,0.068074,-0.112016,-0.055053,0.045375,0.063676,0.016817,-0.061693,-0.026442,-0.031120,0.191916,...,-0.073135,0.135709,0.006571,0.004580,0.166603,0.121948,-0.031010,-0.095062,0.007649,-0.106070
4,0.040953,-0.028588,-0.063608,0.046010,0.081183,-0.021608,-0.077216,0.027718,-0.030726,0.168264,...,-0.064928,0.073787,-0.000367,0.074108,0.142589,0.093344,0.004230,-0.099472,-0.069675,-0.046723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10910,0.025258,-0.056760,-0.053916,-0.080924,0.025728,-0.042500,0.016543,-0.056844,-0.068292,0.035164,...,-0.116704,0.099249,0.008260,0.035744,0.203707,-0.003986,0.023136,-0.156268,-0.018444,-0.033398
10911,0.122435,-0.154742,-0.159965,-0.032322,0.095427,-0.047862,0.000538,-0.144428,-0.148185,0.052159,...,-0.061774,0.067999,0.021077,0.097443,0.181980,0.091247,0.007482,-0.320918,-0.028273,-0.004733
10912,0.079171,-0.031492,-0.051477,-0.086577,0.101841,0.017272,-0.046363,-0.008180,-0.058952,0.015870,...,-0.013864,0.051082,-0.004890,0.170762,0.141724,0.000773,-0.022615,-0.110620,0.037668,-0.070821
10913,0.015078,-0.098757,-0.091811,-0.046112,0.102823,0.005763,-0.019206,-0.023700,-0.079233,0.044035,...,-0.014461,0.088623,0.036835,0.127098,0.156737,-0.003581,-0.047630,-0.176579,0.055320,-0.069404


# BUILDING OUR KERAS MODEL

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2,random_state=42)

In [21]:
X_train.shape

(8732, 200)

In [22]:
y_train

,0
6365,14
157,2
6965,6
6350,14
1441,6
...,...
5734,10
5191,2
5390,10
860,16


In [23]:
y_train.values

array([[14],
       [ 2],
       [ 6],
       ...,
       [10],
       [16],
       [14]])

In [30]:
X_train = X_train.values

In [31]:
X_train.shape

(8732, 200)

In [32]:
y_train1 = to_categorical(y_train)

In [33]:
y_test1 = to_categorical(y_test)

In [34]:
y_train1

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [35]:
X_val = X_train[7859:]
y_val = y_train1[7859:]

## MODEL

In [43]:
from tensorflow.keras.layers import LSTM , Dropout , SpatialDropout1D

In [37]:
in_dim = 200

In [38]:
jazz_size = 32

In [39]:
tot_layers = 600

In [40]:
(tot_layers//3)*2

400

In [48]:
n_most_common_words = 15
emb_dim = 200

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [78]:
from tensorflow.keras.layers import Flatten , Input , Conv2D , MaxPooling2D , BatchNormalization

In [154]:
model10 = Sequential()
model10.add(Dense(100,input_dim=in_dim,activation = "leaky_relu"))
model10.add(Dropout(0.2))
model10.add(Dense(80,activation = "leaky_relu"))
model10.add(Dense(40,activation = "leaky_relu"))
model10.add(Dense(25,activation = "leaky_relu"))
model10.add(Dense(17,activation = "softmax"))

In [155]:
opt = tf.keras.optimizers.AdamW(learning_rate =0.007 , beta_1=0.9,
    beta_2=0.999,
    use_ema=True,
    ema_momentum=0.99)

In [156]:
model10.compile(optimizer = opt , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [206]:
model10.fit(X_train[:7859] , y_train1[:7859] , batch_size = 64 , epochs = 10 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/10
123/123 [==============================] - 0s 4ms/step - loss: 0.6319 - accuracy: 0.9758 - precision: 0.8340 - recall: 0.7356 - val_loss: 1.1769 - val_accuracy: 0.9671 - val_precision: 0.7516 - val_recall: 0.6586
Epoch 2/10
123/123 [==============================] - 0s 3ms/step - loss: 0.6381 - accuracy: 0.9761 - precision: 0.8363 - recall: 0.7372 - val_loss: 1.1220 - val_accuracy: 0.9662 - val_precision: 0.7428 - val_recall: 0.6518
Epoch 3/10
123/123 [==============================] - 0s 3ms/step - loss: 0.6306 - accuracy: 0.9762 - precision: 0.8393 - recall: 0.7371 - val_loss: 1.1143 - val_accuracy: 0.9667 - val_precision: 0.7510 - val_recall: 0.6495
Epoch 4/10
123/123 [==============================] - 0s 3ms/step - loss: 0.6193 - accuracy: 0.9762 - precision: 0.8399 - recall: 0.7362 - val_loss: 1.1683 - val_accuracy: 0.9670 - val_precision: 0.7477 - val_recall: 0.6621
Epoch 5/10
123/123 [==============================] - 0s 3ms/step - loss: 0.6384 - accuracy: 0.9762 - pr

In [207]:
model10.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 100)               20100     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_11 (Dense)            (None, 80)                8080      
                                                                 
 dense_12 (Dense)            (None, 40)                3240      
                                                                 
 dense_13 (Dense)            (None, 25)                1025      
                                                                 
 dense_14 (Dense)            (None, 17)                442       
                                                                 
Total params: 32887 (128.46 KB)
Trainable params: 3288

In [208]:
#scores = model.evaluate(X_test, y_test1)

#for i, m in enumerate(model.metrics_names):
 #   print("\n%s: %.3f"% (m, scores[i]))

In [209]:
predictions = np.argmax(model10.predict(X_test), axis=-1)

69/69 [==============================] - 0s 2ms/step


In [210]:
predictions

array([14,  1, 16, ..., 11,  9, 12], dtype=int64)

In [211]:
#len(predictions)

In [212]:
#y_test.shape

In [213]:
y_test.values

array([[14],
       [ 1],
       [16],
       ...,
       [11],
       [14],
       [12]])

In [214]:
print(classification_report(y_test.values , predictions))

              precision    recall  f1-score   support

           0       0.64      0.60      0.62        68
           1       0.78      0.80      0.79       176
           2       0.73      0.76      0.74        97
           3       0.70      0.64      0.67        90
           4       0.71      0.60      0.65        93
           5       0.67      0.78      0.72       108
           6       0.85      0.82      0.83       126
           7       0.74      0.79      0.77       136
           8       0.75      0.68      0.72       122
           9       0.81      0.75      0.78       155
          10       0.74      0.69      0.71        61
          11       0.66      0.60      0.63       172
          12       0.76      0.84      0.80       182
          13       0.72      0.67      0.69       151
          14       0.63      0.62      0.63       200
          15       0.83      0.91      0.86       169
          16       0.81      0.96      0.88        77

    accuracy              

In [215]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions, average='macro')))
print('F1:', f1_score(y_test.values, predictions, average='macro'))

Precision is 0.7367116931285698
Recall is 0.7361486573527378
F1: 0.7348066061695032


In [205]:
# 40 epanalipseis --->74% accuracy

# Building a different w2vec model to really see what is the root of evil on our 70% accuracy

In [216]:
model_w2v2= Word2Vec(sentences=corpus_full, vector_size=500, window=4, min_count=15, sg = 1 , hs = 1)  # skipgram architecture

In [217]:
model_w2v2.save("word2vec.i_dont_even_know_how_shitty_this_model_is")

In [218]:
def vectorizer2():
    
    
    # Creating a list for storing the vectors ('Description' into vectors)
    #global word_embeddings
    word_embeddings = []
    i = 0
    # Reading the each 'Description'
    for line in io['Cleaned']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in model_w2v2.wv.key_to_index:
                count += 1
                if avgword2vec is None:
                    avgword2vec = model_w2v2.wv[word]
                else:
                    avgword2vec = avgword2vec + model_w2v2.wv[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
            word_embeddings.append(avgword2vec)
        else:
            print("I found it , the error occurs at line:" , i)
          
        i +=1 

    return(pd.DataFrame(word_embeddings))  # Returning our Data as a Dataframe (aesthetic reasons)

In [219]:
X2 = vectorizer2()

In [220]:
X_train, X_test, y_train, y_test = train_test_split(X2.values, y1, test_size=0.2,random_state=42)

In [221]:
y_train1 = to_categorical(y_train)

In [222]:
y_test1 = to_categorical(y_test)

In [223]:
X_val = X_train[7859:]
y_val = y_train1[7859:]

## KERAS TIME

In [229]:
model = Sequential()
model.add(Dense(100,input_dim=500,activation = "leaky_relu"))
model.add(Dropout(0.2))
model.add(Dense(80,activation = "leaky_relu"))
model.add(Dense(40,activation = "leaky_relu"))
model.add(Dense(25,activation = "leaky_relu"))
model.add(Dense(17,activation = "softmax"))

In [230]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 100)               50100     
                                                                 
 dropout_4 (Dropout)         (None, 100)               0         
                                                                 
 dense_21 (Dense)            (None, 80)                8080      
                                                                 
 dense_22 (Dense)            (None, 40)                3240      
                                                                 
 dense_23 (Dense)            (None, 25)                1025      
                                                                 
 dense_24 (Dense)            (None, 17)                442       
                                                                 
Total params: 62887 (245.65 KB)
Trainable params: 6288

In [231]:
opt = tf.keras.optimizers.AdamW(learning_rate =0.007 , beta_1=0.9,
    beta_2=0.999,
    use_ema=True,
    ema_momentum=0.99)

In [232]:
model.compile(optimizer = opt , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [249]:
model.fit(X_train[:7859] , y_train1[:7859] , batch_size = 64 , epochs = 10 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/10
123/123 [==============================] - 1s 4ms/step - loss: 0.5686 - accuracy: 0.9789 - precision: 0.8582 - recall: 0.7687 - val_loss: 1.1762 - val_accuracy: 0.9674 - val_precision: 0.7503 - val_recall: 0.6678
Epoch 2/10
123/123 [==============================] - 0s 4ms/step - loss: 0.5641 - accuracy: 0.9793 - precision: 0.8592 - recall: 0.7744 - val_loss: 1.2417 - val_accuracy: 0.9671 - val_precision: 0.7437 - val_recall: 0.6712
Epoch 3/10
123/123 [==============================] - 0s 3ms/step - loss: 0.5713 - accuracy: 0.9781 - precision: 0.8486 - recall: 0.7633 - val_loss: 1.2177 - val_accuracy: 0.9664 - val_precision: 0.7385 - val_recall: 0.6632
Epoch 4/10
123/123 [==============================] - 0s 3ms/step - loss: 0.5585 - accuracy: 0.9790 - precision: 0.8566 - recall: 0.7724 - val_loss: 1.1978 - val_accuracy: 0.9693 - val_precision: 0.7693 - val_recall: 0.6838
Epoch 5/10
123/123 [==============================] - 0s 3ms/step - loss: 0.5759 - accuracy: 0.9780 - pr

In [250]:
predictions = np.argmax(model.predict(X_test), axis=-1)

69/69 [==============================] - 0s 2ms/step


In [251]:
predictions

array([14,  1, 16, ..., 11,  9, 12], dtype=int64)

In [252]:
print(classification_report(y_test.values , predictions))

              precision    recall  f1-score   support

           0       0.68      0.62      0.65        68
           1       0.76      0.84      0.80       176
           2       0.67      0.79      0.73        97
           3       0.69      0.64      0.67        90
           4       0.71      0.61      0.66        93
           5       0.71      0.76      0.73       108
           6       0.84      0.79      0.81       126
           7       0.74      0.81      0.77       136
           8       0.75      0.74      0.74       122
           9       0.82      0.73      0.77       155
          10       0.71      0.74      0.73        61
          11       0.71      0.59      0.64       172
          12       0.76      0.81      0.79       182
          13       0.69      0.67      0.68       151
          14       0.66      0.65      0.65       200
          15       0.87      0.90      0.88       169
          16       0.85      0.97      0.91        77

    accuracy              

In [253]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions, average='macro')))
print('F1:', f1_score(y_test.values, predictions, average='macro'))

Precision is 0.7418189155522188
Recall is 0.7444507039550632
F1: 0.7413394435123317


In [238]:
# For 10 epochs the above scored 73% accuracy

In [254]:
# For 30 epochs the above scored 74% accuracy

## Model 2

In [316]:
model2 = Sequential()
model2.add(Dense(100,input_dim=500,activation = "leaky_relu"))
model2.add(Dense(180,activation = "leaky_relu"))
model2.add(Dropout(0.2))
model2.add(Dense(60,activation = "leaky_relu"))
#model2.add(Dense(30,activation = "leaky_relu"))
model2.add(Dense(17,activation = "softmax"))

In [317]:
opt = tf.keras.optimizers.AdamW(learning_rate =0.007 , beta_1=0.9,
    beta_2=0.999,
    use_ema=True,
    ema_momentum=0.99)

In [318]:
model2.compile(optimizer = opt , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [325]:
model2.fit(X_train[:7859] , y_train1[:7859] , batch_size = 64 , epochs = 10 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/10
123/123 [==============================] - 1s 4ms/step - loss: 0.6890 - accuracy: 0.9749 - precision: 0.8340 - recall: 0.7168 - val_loss: 1.0679 - val_accuracy: 0.9682 - val_precision: 0.7663 - val_recall: 0.6609
Epoch 2/10
123/123 [==============================] - 0s 4ms/step - loss: 0.6792 - accuracy: 0.9750 - precision: 0.8303 - recall: 0.7234 - val_loss: 1.1319 - val_accuracy: 0.9675 - val_precision: 0.7589 - val_recall: 0.6564
Epoch 3/10
123/123 [==============================] - 0s 4ms/step - loss: 0.6572 - accuracy: 0.9751 - precision: 0.8340 - recall: 0.7210 - val_loss: 1.1288 - val_accuracy: 0.9676 - val_precision: 0.7593 - val_recall: 0.6575
Epoch 4/10
123/123 [==============================] - 0s 4ms/step - loss: 0.6285 - accuracy: 0.9768 - precision: 0.8444 - recall: 0.7431 - val_loss: 1.1321 - val_accuracy: 0.9683 - val_precision: 0.7627 - val_recall: 0.6701
Epoch 5/10
123/123 [==============================] - 0s 4ms/step - loss: 0.6421 - accuracy: 0.9769 - pr

In [326]:
predictions = np.argmax(model2.predict(X_test), axis=-1)

predictions

69/69 [==============================] - 0s 2ms/step


array([14,  1, 16, ..., 11,  9, 12], dtype=int64)

In [327]:
print(classification_report(y_test.values , predictions))

              precision    recall  f1-score   support

           0       0.69      0.62      0.65        68
           1       0.78      0.82      0.80       176
           2       0.69      0.74      0.71        97
           3       0.72      0.63      0.67        90
           4       0.76      0.61      0.68        93
           5       0.68      0.81      0.74       108
           6       0.85      0.83      0.84       126
           7       0.74      0.78      0.76       136
           8       0.77      0.72      0.74       122
           9       0.81      0.71      0.76       155
          10       0.78      0.75      0.77        61
          11       0.66      0.63      0.64       172
          12       0.79      0.80      0.79       182
          13       0.68      0.70      0.69       151
          14       0.67      0.67      0.67       200
          15       0.86      0.89      0.88       169
          16       0.84      0.99      0.90        77

    accuracy              

In [328]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions, average='macro')))
print('F1:', f1_score(y_test.values, predictions, average='macro'))

Precision is 0.7497295033663197
Recall is 0.7478797082622529
F1: 0.7468548054573868


In [324]:
# 74% for 10 epochs (model2)

In [329]:
# 75% for 20 epochs

### Model 3

In [372]:
model6 = Sequential()
model6.add(Dense(100,input_dim=500,activation = "leaky_relu"))
model6.add(Dense(180,activation = "leaky_relu"))
model6.add(Dropout(0.2))
model6.add(Dense(60,activation = "leaky_relu"))
#model2.add(Dense(30,activation = "leaky_relu"))
model6.add(Dense(17,activation = "softmax"))

In [373]:
opt = tf.keras.optimizers.AdamW(learning_rate =0.007 , beta_1=0.9,
    beta_2=0.999,
    use_ema=True,
    ema_momentum=0.99)

In [374]:
model6.compile(optimizer = opt , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [379]:
model6.fit(X_train[:7859] , y_train1[:7859] , batch_size = 32 , epochs = 5 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/5
246/246 [==============================] - 1s 3ms/step - loss: 0.7565 - accuracy: 0.9733 - precision: 0.8223 - recall: 0.6967 - val_loss: 1.1208 - val_accuracy: 0.9648 - val_precision: 0.7455 - val_recall: 0.6105
Epoch 2/5
246/246 [==============================] - 1s 3ms/step - loss: 0.7484 - accuracy: 0.9731 - precision: 0.8177 - recall: 0.6979 - val_loss: 1.2005 - val_accuracy: 0.9652 - val_precision: 0.7371 - val_recall: 0.6357
Epoch 3/5
246/246 [==============================] - 1s 3ms/step - loss: 0.7429 - accuracy: 0.9732 - precision: 0.8173 - recall: 0.7015 - val_loss: 1.2353 - val_accuracy: 0.9652 - val_precision: 0.7399 - val_recall: 0.6289
Epoch 4/5
246/246 [==============================] - 1s 3ms/step - loss: 0.7114 - accuracy: 0.9740 - precision: 0.8227 - recall: 0.7110 - val_loss: 1.1582 - val_accuracy: 0.9675 - val_precision: 0.7621 - val_recall: 0.6495
Epoch 5/5
246/246 [==============================] - 1s 3ms/step - loss: 0.6689 - accuracy: 0.9756 - precisi

In [380]:
predictions = np.argmax(model6.predict(X_test), axis=-1)

predictions

69/69 [==============================] - 0s 2ms/step


array([14,  1, 16, ..., 11,  9, 12], dtype=int64)

In [381]:
print(classification_report(y_test.values , predictions))

              precision    recall  f1-score   support

           0       0.69      0.60      0.65        68
           1       0.81      0.82      0.82       176
           2       0.72      0.74      0.73        97
           3       0.73      0.63      0.68        90
           4       0.75      0.59      0.66        93
           5       0.74      0.75      0.74       108
           6       0.83      0.83      0.83       126
           7       0.76      0.78      0.77       136
           8       0.81      0.67      0.74       122
           9       0.84      0.70      0.77       155
          10       0.68      0.70      0.69        61
          11       0.66      0.67      0.66       172
          12       0.77      0.82      0.79       182
          13       0.68      0.70      0.69       151
          14       0.61      0.70      0.65       200
          15       0.84      0.89      0.87       169
          16       0.84      0.97      0.90        77

    accuracy              

In [382]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions, average='macro')))
print('F1:', f1_score(y_test.values, predictions, average='macro'))

Precision is 0.7515128800775556
Recall is 0.7414660237509374
F1: 0.7441367038194585


In [383]:
# 15 epochs , batch size = 32 , adamw